## Importing the unprocessed consolidatedfile  

In [2]:
import boto3
from boto.s3.connection import S3Connection
import os
import json
import boto.s3
import sys
import datetime
from boto.s3.key import Key
from pprint import pprint
import pandas as pd
import urllib
import csv
import io
import requests
import time
import json
import datetime
from pprint import pprint
import scipy
import numpy as np

In [3]:
svcgdata = pd.read_csv('downloaded_samples/unprocessed_consolidated_sample_svcg_file.csv', delimiter=",", header=None, index_col=False,
                             names=["LoanSequenceNumber", "MonthlyReportingPeriod",
                                    "CurrentActualUPB", "CurrentLonDeliquencyStatus",
                                    "LoanAge", "RemainingMonthsForLegalMaturity",
                                    "RepurchaseFlag", "ModificationFlag",
                                    "ZeroBalanceCode", "ZeroBalanceEffectiveDate",
                                    "CurrentInterestRate", "CurrentDeferredUPB",
                                    "DDLPI", "MIRecoveries", "NetSalesProceeds",
                                    "NonMIRecoveries", "Expenses", "LegalCosts",
                                    "MaintenanceAndPreservationCosts", "TaxesAndInsurance",
                                    "MiscellaneousExpenses", "ActualLossCalculation",
                                    "ModificationCost"])


    
print("Shape of the initial svcg file"svcgdata.shape)

C:\Users\sneha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (3,7,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(24798279, 23)


## Removing rows from the dataframe 

In [4]:
svcgdata=svcgdata[["LoanSequenceNumber",
                                    "CurrentActualUPB", "CurrentLonDeliquencyStatus",
                                    "LoanAge", "RemainingMonthsForLegalMaturity",
                                    "ModificationFlag",
                                    "ZeroBalanceCode", "ZeroBalanceEffectiveDate",
                                    "CurrentInterestRate", "CurrentDeferredUPB",
                                    "MIRecoveries", "NetSalesProceeds",
                                    "NonMIRecoveries", "Expenses", "LegalCosts",
                                    "MaintenanceAndPreservationCosts", "TaxesAndInsurance",
                                    "ActualLossCalculation",
                                    "ModificationCost"]]
    
print(svcgdata.shape)

(24798279, 19)


## Removing values if loan sequence number doesnt exist 


In [6]:
svcgdata.dropna(subset=['LoanSequenceNumber'], inplace=True)
print(svcgdata.shape)

(24798279, 19)


In [40]:
svcgdata.dropna(subset=['CurrentInterestRate'], inplace=True)
print(svcgdata.shape)

(24798279, 19)


## Filling NaN values 

In [5]:
# svcgdata['CurrentLonDeliquencyStatus'].replace('xx', '0', inplace=True)
svcgdata['CurrentLonDeliquencyStatus'].loc[(svcgdata['CurrentLonDeliquencyStatus']=='XX')] = 0
svcgdata['CurrentLonDeliquencyStatus'].loc[(svcgdata['CurrentLonDeliquencyStatus']=='R')] = 0

print(svcgdata.groupby('CurrentLonDeliquencyStatus')['LoanSequenceNumber'].count())

C:\Users\sneha\Anaconda3\lib\site-packages\pandas\core\indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


CurrentLonDeliquencyStatus
0     8416120
1       25816
2        4693
3        1826
4        1058
5         845
6         619
7         493
8         394
9         312
10        266
11        225
12        185
13        171
14        130
15        116
16         97
17         89
18         74
19         61
20         51
21         47
22         45
23         44
24         42
25         39
26         27
27         25
28         20
29         17
       ...   
72        177
73        158
74        143
75        123
76        112
77        102
78         96
79         91
8       20527
80         85
81         75
82         69
83         61
84         59
85         49
86         46
87         36
88         33
89         27
9       18247
90         26
91         23
92         23
93         15
94         13
95         12
96         10
97          6
98          5
99          4
Name: LoanSequenceNumber, dtype: int64


In [6]:
svcgdata['ModificationFlag'].loc[(svcgdata['ModificationFlag']== 0)] = 'N'
svcgdata[['ModificationFlag']]= svcgdata[['ModificationFlag']].fillna('N')
print(svcgdata.groupby('ModificationFlag')['LoanSequenceNumber'].count())

ModificationFlag
N    24787169
Y       11110
Name: LoanSequenceNumber, dtype: int64


In [7]:
print(svcgdata.groupby('ZeroBalanceCode')['LoanSequenceNumber'].count())

svcgdata[['ZeroBalanceCode']]= svcgdata[['ZeroBalanceCode']].fillna('0')
svcgdata['ModificationFlag'].loc[(svcgdata['ModificationFlag']== 'N')] = 0

print(svcgdata.groupby('ZeroBalanceCode')['LoanSequenceNumber'].count())

ZeroBalanceCode
1.0    314001
3.0      5799
6.0      1605
9.0      7638
Name: LoanSequenceNumber, dtype: int64


C:\Users\sneha\Anaconda3\lib\site-packages\pandas\core\indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


ZeroBalanceCode
1.0      314001
3.0        5799
6.0        1605
9.0        7638
0      24469236
Name: LoanSequenceNumber, dtype: int64


In [8]:
svcgdata[['ZeroBalanceEffectiveDate']]= svcgdata[['ZeroBalanceEffectiveDate']].fillna('196901')
svcgdata[['MIRecoveries']]= svcgdata[['MIRecoveries']].fillna(0)
svcgdata[['NetSalesProceeds']]= svcgdata[['NetSalesProceeds']].fillna('U')
svcgdata[['NonMIRecoveries']]= svcgdata[['NonMIRecoveries']].fillna(0)
svcgdata[['Expenses']]= svcgdata[['Expenses']].fillna(0)
svcgdata[['LegalCosts']]= svcgdata[['LegalCosts']].fillna(0)
svcgdata[['MaintenanceAndPreservationCosts']]= svcgdata[['MaintenanceAndPreservationCosts']].fillna(0)
svcgdata[['TaxesAndInsurance']]= svcgdata[['TaxesAndInsurance']].fillna(0)
svcgdata[['ModificationCost']]= svcgdata[['ModificationCost']].fillna(0)
svcgdata[['ActualLossCalculation']]= svcgdata[['ActualLossCalculation']].fillna(0)


svcgdata.head(5)
# print(svcgdata.groupby('ZeroBalanceEffectiveDate')['LoanSequenceNumber'].count())

,LoanSequenceNumber,CurrentActualUPB,CurrentLonDeliquencyStatus,LoanAge,RemainingMonthsForLegalMaturity,ModificationFlag,ZeroBalanceCode,ZeroBalanceEffectiveDate,CurrentInterestRate,CurrentDeferredUPB,MIRecoveries,NetSalesProceeds,NonMIRecoveries,Expenses,LegalCosts,MaintenanceAndPreservationCosts,TaxesAndInsurance,ActualLossCalculation,ModificationCost
0,F116Q1000017,91000.0,0,0,360,0,0,196901,4.0,0.0,0.0,U,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,F116Q1000017,90000.0,0,1,359,0,0,196901,4.0,0.0,0.0,U,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,F116Q1000017,90000.0,0,2,358,0,0,196901,4.0,0.0,0.0,U,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,F116Q1000017,90000.0,0,3,357,0,0,196901,4.0,0.0,0.0,U,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,F116Q1000017,90000.0,0,4,356,0,0,196901,4.0,0.0,0.0,U,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## checking for any NaN values that still exist 

In [9]:
# svcgdata.loc[:, svcgdata.isnull().any()]

""
0
1
2
3
4
5
6
7
8
9


## Keeping all the data types right 

In [10]:
svcgdata[["CurrentLonDeliquencyStatus","LoanAge", "RemainingMonthsForLegalMaturity","ZeroBalanceCode"]]=svcgdata[["CurrentLonDeliquencyStatus",
                                    "LoanAge", "RemainingMonthsForLegalMaturity",
                                    "ZeroBalanceCode"]].astype(int)

svcgdata[["CurrentActualUPB", "CurrentInterestRate", "CurrentDeferredUPB",
                                    "MIRecoveries", "NonMIRecoveries", "Expenses", "LegalCosts",
                                    "MaintenanceAndPreservationCosts", "TaxesAndInsurance",
                                    "ActualLossCalculation",
                                    "ModificationCost"]]=svcgdata[["CurrentActualUPB", "CurrentInterestRate", "CurrentDeferredUPB",
                                    "MIRecoveries", "NonMIRecoveries", "Expenses", "LegalCosts",
                                    "MaintenanceAndPreservationCosts", "TaxesAndInsurance",
                                    "ActualLossCalculation",
                                    "ModificationCost"]].astype(float)


svcgdata[["LoanSequenceNumber", "ModificationFlag", "ZeroBalanceEffectiveDate"]]=svcgdata[["LoanSequenceNumber", "ModificationFlag", "ZeroBalanceEffectiveDate"]].astype('str')

svcgdata.head(5)

,LoanSequenceNumber,CurrentActualUPB,CurrentLonDeliquencyStatus,LoanAge,RemainingMonthsForLegalMaturity,ModificationFlag,ZeroBalanceCode,ZeroBalanceEffectiveDate,CurrentInterestRate,CurrentDeferredUPB,MIRecoveries,NetSalesProceeds,NonMIRecoveries,Expenses,LegalCosts,MaintenanceAndPreservationCosts,TaxesAndInsurance,ActualLossCalculation,ModificationCost
0,F116Q1000017,91000.0,0,0,360,0,0,196901,4.0,0.0,0.0,U,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,F116Q1000017,90000.0,0,1,359,0,0,196901,4.0,0.0,0.0,U,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,F116Q1000017,90000.0,0,2,358,0,0,196901,4.0,0.0,0.0,U,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,F116Q1000017,90000.0,0,3,357,0,0,196901,4.0,0.0,0.0,U,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,F116Q1000017,90000.0,0,4,356,0,0,196901,4.0,0.0,0.0,U,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# df.sort('C').drop_duplicates(subset=['A', 'B'], take_last=True)
#df['Tenant'].replace('', np.nan, inplace=True)

In [11]:
origdata = pd.read_csv('downloaded_samples/unprocessed_consolidated_sample_orig_file.csv', delimiter=",", header=None, index_col=False,
                                  names=["CreditScore", "FirstPaymentDate", "FirstTimeHomeBuyerFlag",
                                         "MaturityDate", "MSA", "MortgageInsurancePercentage",
                                         "NumberOfUnits", "OccupancyStatus",
                                         "OroginalCLTV", "OriginalDTIRatio", "OriginalUPB",
                                         "OriginalLTV", "OriginalInterestRate",
                                         "Channel", "PPMFlag",
                                         "ProductType", "PropertyState", "PropertyType",
                                         "PostalCode", "LoanSequenceNumber", "LoanPurpose",
                                         "OriginalLoanTerm", "NumberOfBorrowers",
                                         "SellerName", "ServiceName",
                                         "SuperConformingFlag"])
    
    
print(origdata.shape)

C:\Users\sneha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (0,5,9,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(575000, 26)


In [12]:
origdata=origdata[["CreditScore", "FirstPaymentDate", "MaturityDate", "MortgageInsurancePercentage",
                                         "OroginalCLTV", "OriginalDTIRatio", "OriginalUPB",
                                         "OriginalLTV", "OriginalInterestRate",
                                         "PPMFlag", "PropertyState", "PropertyType",
                                         "LoanSequenceNumber", "OriginalLoanTerm", "ServiceName"]]
print(origdata.shape )
print (origdata.head(5))

(575000, 15)
  CreditScore  FirstPaymentDate  MaturityDate MortgageInsurancePercentage  \
0         742            201603        204602                           0   
1         776            201604        204603                           0   
2         776            201603        204602                           0   
3         719            201603        204602                          30   
4         695            201603        204602                          30   

   OroginalCLTV OriginalDTIRatio  OriginalUPB  OriginalLTV  \
0          80.0               44        91000         80.0   
1          46.0               47       108000         46.0   
2          75.0               34        59000         75.0   
3          95.0               27       187000         95.0   
4          95.0               38       176000         95.0   

   OriginalInterestRate PPMFlag PropertyState PropertyType LoanSequenceNumber  \
0                 4.000       N            OH           SF       F116Q

## Lets find the rows with NaN values

In [13]:
origdata.loc[:, origdata.isnull().any()]

,OroginalCLTV,OriginalDTIRatio,OriginalLTV,PPMFlag
0,80.0,44,80.0,N
1,46.0,47,46.0,N
2,75.0,34,75.0,N
3,95.0,27,95.0,N
4,95.0,38,95.0,N
5,57.0,48,57.0,N
6,80.0,17,80.0,N
7,70.0,43,70.0,N
8,86.0,31,86.0,N
9,80.0,25,80.0,N


## Let's check the number of NaN values in each column 


In [14]:
origdata.isnull().sum(axis=0)

CreditScore                       0
FirstPaymentDate                  0
MaturityDate                      0
MortgageInsurancePercentage       0
OroginalCLTV                     18
OriginalDTIRatio                268
OriginalUPB                       0
OriginalLTV                      17
OriginalInterestRate              0
PPMFlag                        2612
PropertyState                     0
PropertyType                      0
LoanSequenceNumber                0
OriginalLoanTerm                  0
ServiceName                       0
dtype: int64

In [15]:
origdata = origdata[np.isfinite(origdata['OroginalCLTV'])]
               
origdata.isnull().sum(axis=0)

CreditScore                       0
FirstPaymentDate                  0
MaturityDate                      0
MortgageInsurancePercentage       0
OroginalCLTV                      0
OriginalDTIRatio                268
OriginalUPB                       0
OriginalLTV                       0
OriginalInterestRate              0
PPMFlag                        2611
PropertyState                     0
PropertyType                      0
LoanSequenceNumber                0
OriginalLoanTerm                  0
ServiceName                       0
dtype: int64

In [89]:
# print(origdata.OriginalDTIRatio.dtype)
# # origdata = origdata[np.isfinite(origdata['OriginalDTIRatio'])]
# np.isfinite(origdata.OriginalDTIRatio.values.astype("float64"))
# origdata.isnull().sum(axis=0)
# origdata.OriginalDTIRatio.astype(np.float64)
# origdata = origdata[np.isfinite(origdata['OriginalDTIRatio'])]
# print(origdata.groupby('OroginalCLTV')['LoanSequenceNumber'].count())                
# print(origdata.groupby('OriginalDTIRatio')['LoanSequenceNumber'].count()) 
# origdata[["OroginalCLTV", "OriginalDTIRatio"]]=origdata[["OroginalCLTV", "OriginalDTIRatio"]].astype(float)

In [16]:
# origdata.loc[:, origdata.isnull().any()]

,OriginalDTIRatio,PPMFlag
0,44,N
1,47,N
2,34,N
3,27,N
4,38,N
5,48,N
6,17,N
7,43,N
8,31,N
9,25,N


In [17]:
print(origdata.groupby('OriginalDTIRatio')['LoanSequenceNumber'].count())
print(origdata.groupby('PPMFlag')['LoanSequenceNumber'].count())

OriginalDTIRatio
1.0         4
2.0        14
3.0         9
4.0        11
5.0        25
6.0        43
7.0        54
8.0        85
9.0       134
10.0      175
11.0      244
12.0      360
13.0      420
14.0      471
15.0      593
16.0      775
17.0      896
18.0      979
19.0     1114
20.0     1230
21.0     1329
22.0     1522
23.0     1604
24.0     1757
25.0     1728
26.0     1779
27.0     1848
28.0     1931
29.0     2042
30.0     2126
        ...  
41      15685
42      15391
43      15737
44      15878
45      14901
46       9054
47       8927
48       8755
49       8660
5         372
50       7768
51       3578
52       3391
53       3152
54       3103
55       2746
56       1851
57       1709
58       1640
59       1552
6         507
60       1477
61       1283
62       1144
63       1108
64       1041
65        798
7         708
8        1070
9        1463
Name: LoanSequenceNumber, dtype: int64
PPMFlag
N    572122
Y       249
Name: LoanSequenceNumber, dtype: int64


In [ ]:
print(origdata.groupby('PPMFlag')['LoanSequenceNumber'].count())

In [18]:
origdata[["OriginalDTIRatio"]]=origdata[["OriginalDTIRatio"]].fillna(0)
origdata[["PPMFlag"]]=origdata[["PPMFlag"]].fillna('N')

print(origdata.loc[:, origdata.isnull().any()])
print("The types of flags are :",origdata.groupby('PPMFlag')['LoanSequenceNumber'].count())

print (origdata.head(5))

Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]

[574982 rows x 0 columns]
The types of flags are : PPMFlag
N    574733
Y       249
Name: LoanSequenceNumber, dtype: int64
  CreditScore  FirstPaymentDate  MaturityDate MortgageInsurancePercentage  \
0         742            201603        204602                           0   
1         776            201604        204603                           0   
2         776            201603        204602                           0   
3         719            201603        204602                          30   
4         695            201603        204602                

In [105]:
# print(origdata.groupby('ServiceName')['LoanSequenceNumber'].count())

ServiceName
ABNAMROMTGEGROUP_INC       567
ALLYBANK                  2738
AMERIHOMEMTGECOMPANY      1475
AMTRUSTBANK                117
ARVESTCENTRALMTGECO       2466
BACHOMELOANSERVICING      3740
BANKOFAMERICA_NA         42673
BRANCHBANKING&TRUSTC     24731
CALIBERHOMELOANS_INC      4735
CENTRALMTGECO              693
CITIMORTGAGE_INC         21865
COLONIALSAVINGS_FA         402
COLORADOFEDSAVINGSBA       342
COUNTRYWIDE               3979
DITECHFINANCIALLLC        1174
FIFTHTHIRDBANK            4467
FIFTHTHIRDMTGECO          3434
FIRSTKEYMTGELLC            273
FLAGSTARBANK_FSB           528
FLAGSTARCAPITALMARKE       700
FRANKLINAMERICANMTGE      2862
FREEDOMMTGECORP            134
FREMONTBANK                424
GMACMORTGAGE_LLC          3332
IMPACMTGECORP              140
JPMORGANCHASEBANK_NA     56303
LAKEVIEWLOANSERVICIN      3036
LOANDEPOTCOM_LLC           128
MARIXSERVICINGLLC          401
MATRIXFINANCIALSERVI       891
METLIFEHOMELOANS_ADI      2042
NATIONSTARMTGELLC        15

## Saving the dataframe in a csv file

In [19]:
origdata.to_csv('origdata.csv', index=False)
print ("successfully saved Original data as a CSV file")

svcgdata.to_csv('svcgdata.csv', index=False)
print ("successfully saved SVCG data as a CSV file")

successfully saved Original data as a CSV file
successfully saved SVCG data as a CSV file


## Saving the file on s3

In [ ]:
# # getting the csv with secret key and access key 

# with open('config.json') as data_file:    
#     data = json.load(data_file)
    

    
# # secret keys 

# AWSAccess1=data["AWSAccess"]
# AWSSecret1=data["AWSSecret"]

# c = boto.connect_s3(AWSAccess1, AWSSecret1)
# conn = S3Connection(AWSAccess1, AWSSecret1)    
# bucket = c.get_bucket('team8midterm')
# b = c.get_bucket(bucket, validate=False)

# b = c.get_bucket(bucket, validate=False)

# k=Key(bucket)
# k.key="svcgdata.csv"
# url = "https://s3.amazonaws.com/team8midterm/"
# r = requests.get(url)

# k = Key(b)
# k.key ='svcgdata.csv'

# # k.set_contents_from_filename("wrangleddata.csv")
# k.content_type = r.headers['content-type']
# k.set_contents_from_filename('svcgdata.csv')
# print('successfully uploaded svcgdata.csv to s3')

# k=Key(bucket)
# k.key="origdata.csv"
# url = "https://s3.amazonaws.com/team8midterm/"
# r = requests.get(url)

# k = Key(b)
# k.key ='origdata.csv'

# # k.set_contents_from_filename("wrangleddata.csv")
# k.content_type = r.headers['content-type']
# k.set_contents_from_filename('origdata.csv')
# print('successfully uploaded origdata.csv to s3')